The following codes  
https://towardsdatascience.com/selenium-tweepy-to-scrap-tweets-from-tweeter-and-analysing-sentiments-1804db3478ac
https://www.earthdatascience.org/courses/earth-analytics-python/using-apis-natural-language-processing-twitter/get-and-use-twitter-data-in-python/  
https://medium.freecodecamp.org/how-to-make-your-own-sentiment-analyzer-using-python-and-googles-natural-language-api-9e91e1c493e  
https://www.programcreek.com/python/example/76301/tweepy.Cursor


In [30]:
# !pip3 install tweepy

In [16]:
import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)
import twitterkeys
import tweepy
from tweepy import OAuthHandler

In [8]:
consumer_key = twitterkeys.CONSUMER_KEY
consumer_secret   = twitterkeys.CONSUMER_SECRET
access_token   = twitterkeys.ACCESS_TOKEN
access_token_secret   = twitterkeys.ACCESS_TOKEN_SECRET

In [9]:
def authentication(cons_key, cons_secret, acc_token, acc_secret):
    auth = tweepy.OAuthHandler(cons_key, cons_secret)
    auth.set_access_token(acc_token, acc_secret)
    api = tweepy.API(auth)
    return api

In [10]:
from datetime import datetime, timedelta
today_datetime = datetime.today().now()
yesterday_datetime = today_datetime - timedelta(days=1)
today_date = today_datetime.strftime('%Y-%m-%d')
yesterday_date = yesterday_datetime.strftime('%Y-%m-%d')

In [55]:
def search_tweets(keyword, total_tweets):
    today_datetime = datetime.today().now()
    yesterday_datetime = today_datetime - timedelta(days=1)
    today_date = today_datetime.strftime('%Y-%m-%d')
    yesterday_date = yesterday_datetime.strftime('%Y-%m-%d')
    api = authentication(consumer_key,consumer_secret,access_token,access_token_secret)
    search_result = tweepy.Cursor(api.search, 
                                  q=keyword, 
                                  since=yesterday_date, 
                                  result_type='recent', 
                                  tweet_mode='extended',
                                  lang='en').items(total_tweets)
    
    

    return search_result


In [61]:
test = search_tweets("'angry' and 'happy'",1)
for i in test:
    print(i.full_text)
    print(" ")
    print(i)


RT @eminus: Happy to report that testing out the @Oculus Quest in flight on a 737 works just fine.  Stationary guardian and playtested Spac…
 
Status(metadata={'iso_language_code': 'en', 'result_type': 'recent'}, geo=None, retweeted_status=Status(metadata={'iso_language_code': 'en', 'result_type': 'recent'}, geo=None, place=None, id_str='1131589416219058176', source='Twitter for Android', lang='en', extended_entities={'media': [{'expanded_url': 'https://twitter.com/eminus/status/1131589416219058176/photo/1', 'media_url_https': 'https://pbs.twimg.com/media/D7Q2grBUYAAfnfC.jpg', 'type': 'photo', 'id': 1131589341308805120, 'sizes': {'large': {'w': 1280, 'resize': 'fit', 'h': 720}, 'medium': {'w': 1200, 'resize': 'fit', 'h': 675}, 'thumb': {'w': 150, 'resize': 'crop', 'h': 150}, 'small': {'w': 680, 'resize': 'fit', 'h': 383}}, 'id_str': '1131589341308805120', 'media_url': 'http://pbs.twimg.com/media/D7Q2grBUYAAfnfC.jpg', 'url': 'https://t.co/794isCCrxc', 'display_url': 'pic.twitter.com/794

RT @sftwqian: I’m so in love with Kun, and it’s not that “oh let me marry him” type shit like, I just really care about him, I always want…


In [38]:
def search_tweets(keyword, total_tweets):
    today_datetime = datetime.today().now()
    yesterday_datetime = today_datetime - timedelta(days=1)
    today_date = today_datetime.strftime('%Y-%m-%d')
    yesterday_date = yesterday_datetime.strftime('%Y-%m-%d')
    api = authentication(consumer_key,consumer_secret,access_token,access_token_secret)
    search_result = tweepy.Cursor(api.search, 
                                  q=keyword, 
                                  since=yesterday_date, 
                                  result_type='recent', 
                                  lang='en').items(total_tweets)
    
    listOfTweets = []
    
    for tweet in search_result:
        # Add tweets in this format
        dict_ = {'Screen Name': tweet.user.screen_name,
                'User Name': tweet.user.name,
                'Tweet Created At': tweet.created_at,
                'Tweet Text': tweet.text,
                'User Location': tweet.user.location,
                'Tweet Coordinates': tweet.coordinates,
                'Retweet Count':tweet.retweet_count,
                'Retweeted': tweet.retweeted,
                'Phone Type': tweet.source,
                'Favorite Count': tweet.favorite_count,
                'Favorited': tweet.favorited,
                'Replied': tweet.in_reply_to_status_id_str
                }
        listOfTweets.append(dict_)   
    return listOfTweets


In [39]:
test = search_tweets(('angry','happy'),1)

In [40]:
test

[{'Favorite Count': 0,
  'Favorited': False,
  'Phone Type': 'Twitter Web Client',
  'Replied': '1131825548110360576',
  'Retweet Count': 0,
  'Retweeted': False,
  'Screen Name': 'SylDLucas',
  'Tweet Coordinates': None,
  'Tweet Created At': datetime.datetime(2019, 5, 24, 11, 41, 40),
  'Tweet Text': '@ChildfreeTanya Would there be a backlash, though? Would people actually be angry if Penny and Leonard just ended u… https://t.co/H9QX8eW9iO',
  'User Location': 'Connecticut',
  'User Name': 'Sylvia D. Lucas'}]

In [33]:
def get_tweets(listOfTweets, keyword, numOfTweets):
    # Iterate through all tweets containing the given word, api search mode
    for tweet in tweepy.Cursor(api.search, q=keyword).items(numOfTweets):
        # Add tweets in this format
        dict_ = {'Screen Name': tweet.user.screen_name,
                'User Name': tweet.user.name,
                'Tweet Created At': unicode(tweet.created_at),
                'Tweet Text': tweet.text,
                'User Location': unicode(tweet.user.location),
                'Tweet Coordinates': unicode(tweet.coordinates),
                'Retweet Count': unicode(tweet.retweet_count),
                'Retweeted': unicode(tweet.retweeted),
                'Phone Type': unicode(tweet.source),
                'Favorite Count': unicode(tweet.favorite_count),
                'Favorited': unicode(tweet.favorited),
                'Replied': unicode(tweet.in_reply_to_status_id_str)
                }
        listOfTweets.append(dict_)   
    return listOfTweets

# Connect to DB 

Status(metadata={'iso_language_code': 'en', 'result_type': 'recent'}, geo=None, retweeted_status=Status(metadata={'iso_language_code': 'en', 'result_type': 'recent'}, geo=None, place=None, id_str='1131880486685560832', source='Twitter Web Client', lang='en', in_reply_to_user_id=None, text="Happy with @JustinTrudeau's government: \nOmar Khadr\nAngry with Justin Trudeau's government: \nMark Norman, Jody Wils… https://t.co/1tnCdYd6uk", source_url='http://twitter.com', truncated=True, entities={'urls': [{'expanded_url': 'https://twitter.com/i/web/status/1131880486685560832', 'url': 'https://t.co/1tnCdYd6uk', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [117, 140]}], 'hashtags': [], 'symbols': [], 'user_mentions': [{'id_str': '14260960', 'screen_name': 'JustinTrudeau', 'id': 14260960, 'name': 'Justin Trudeau', 'indices': [11, 25]}]}, in_reply_to_screen_name=None, in_reply_to_status_id=None, id=1131880486685560832, coordinates=None, retweeted=False, retweet_count=4, author=User(h

In [ ]:

class TwitterClient(object): 
    def __init__(self):
        try: 
            # OAuthHandler object 
            auth = OAuthHandler(consumer_key, consumer_secret) 
            # set access token and secret 
            auth.set_access_token(access_token, access_token_secret) 
            # create tweepy API object to fetch tweets 
            self.api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
            
        except tweepy.TweepError as e:
            print(f"Error: Twitter Authentication Failed - \n{str(e)}") 

    # Function to fetch tweets
    def get_tweets(self, query, maxTweets = 1000): 
        # empty list to store parsed tweets 
        tweets = [] 
        sinceId = None
        max_id = -1
        tweetCount = 0
        tweetsPerQry = 100
        
        while tweetCount < maxTweets:
            try:
                if (max_id <= 0):
                    if (not sinceId):
                        new_tweets = self.api.search(q=query, count=tweetsPerQry)
                    else:
                        new_tweets = self.api.search(q=query, count=tweetsPerQry,
                                                since_id=sinceId)
                else:
                    if (not sinceId):
                        new_tweets = self.api.search(q=query, count=tweetsPerQry,
                                                max_id=str(max_id - 1))
                    else:
                        new_tweets = self.api.search(q=query, count=tweetsPerQry,
                                                max_id=str(max_id - 1),
                                                since_id=sinceId)
                if not new_tweets:
                    print("No more tweets found")
                    break
                    
                for tweet in new_tweets:
                    parsed_tweet = {} 
                    parsed_tweet['tweets'] = tweet.text 

                    # appending parsed tweet to tweets list 
                    if tweet.retweet_count > 0: 
                        # if tweet has retweets, ensure that it is appended only once 
                        if parsed_tweet not in tweets: 
                            tweets.append(parsed_tweet) 
                    else: 
                        tweets.append(parsed_tweet) 
                        
                tweetCount += len(new_tweets)
                print("Downloaded {0} tweets".format(tweetCount))
                max_id = new_tweets[-1].id

            except tweepy.TweepError as e:
                print("Tweepy error : " + str(e))
                break
        
        return pd.DataFrame(tweets)